In [1]:
import pandas as pd

In [80]:
v = pd.read_csv('vehicles.csv')

In [81]:
v.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [82]:
v = v.dropna()
v.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
126,7305672709,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,68472.0,clean,automatic,1GCWGAFP8J1309579,rwd,full-size,van,white,al
127,7305672266,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,gas,69125.0,clean,automatic,1GCWGAFP4K1214373,rwd,full-size,van,white,al
128,7305672252,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,66555.0,clean,automatic,1GCWGAFPXJ1337903,rwd,full-size,van,white,al
215,7316482063,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,gas,155000.0,clean,automatic,JTDBT123520243495,fwd,compact,sedan,blue,al
219,7316429417,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,gas,110661.0,clean,automatic,WBAHD6322SGK86772,rwd,mid-size,sedan,white,al


In [83]:
## convert categorical variables to set membership indicators
## and GridSearchCV Polynomial Features and Ridge Regression pipline

In [84]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [85]:
v.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34868 entries, 126 to 426836
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            34868 non-null  int64  
 1   region        34868 non-null  object 
 2   price         34868 non-null  int64  
 3   year          34868 non-null  float64
 4   manufacturer  34868 non-null  object 
 5   model         34868 non-null  object 
 6   condition     34868 non-null  object 
 7   cylinders     34868 non-null  object 
 8   fuel          34868 non-null  object 
 9   odometer      34868 non-null  float64
 10  title_status  34868 non-null  object 
 11  transmission  34868 non-null  object 
 12  VIN           34868 non-null  object 
 13  drive         34868 non-null  object 
 14  size          34868 non-null  object 
 15  type          34868 non-null  object 
 16  paint_color   34868 non-null  object 
 17  state         34868 non-null  object 
dtypes: float64(2), int64(2), obj

In [86]:
list(v.columns)

['id',
 'region',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'VIN',
 'drive',
 'size',
 'type',
 'paint_color',
 'state']

In [87]:
v.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
126,7305672709,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,68472.0,clean,automatic,1GCWGAFP8J1309579,rwd,full-size,van,white,al
127,7305672266,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,gas,69125.0,clean,automatic,1GCWGAFP4K1214373,rwd,full-size,van,white,al
128,7305672252,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,66555.0,clean,automatic,1GCWGAFPXJ1337903,rwd,full-size,van,white,al
215,7316482063,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,gas,155000.0,clean,automatic,JTDBT123520243495,fwd,compact,sedan,blue,al
219,7316429417,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,gas,110661.0,clean,automatic,WBAHD6322SGK86772,rwd,mid-size,sedan,white,al


In [88]:
def cat_to_memind(L):
  L = [str(i) for i in list(L)]
  e = set(L)
  M = []
  for ee in e:
    E = []
    for j in L:
      if j == ee:
        E.append(1)
      else:
        E.append(0)
    M.append((E, ee))
  return(tuple(M))

In [89]:
v.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
126,7305672709,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,68472.0,clean,automatic,1GCWGAFP8J1309579,rwd,full-size,van,white,al
127,7305672266,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,gas,69125.0,clean,automatic,1GCWGAFP4K1214373,rwd,full-size,van,white,al
128,7305672252,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,66555.0,clean,automatic,1GCWGAFPXJ1337903,rwd,full-size,van,white,al
215,7316482063,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,gas,155000.0,clean,automatic,JTDBT123520243495,fwd,compact,sedan,blue,al
219,7316429417,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,gas,110661.0,clean,automatic,WBAHD6322SGK86772,rwd,mid-size,sedan,white,al


In [90]:
v1 = v.drop(['manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel','region','title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'state', 'VIN','id'], axis=1)

In [91]:
v1.head()

,price,year,odometer
126,0,2018.0,68472.0
127,0,2019.0,69125.0
128,0,2018.0,66555.0
215,4000,2002.0,155000.0
219,2500,1995.0,110661.0


In [56]:
len(list(v1.columns))

3

In [57]:
list(v1.columns)[1:]

['year', 'odometer']

In [58]:
pipe = Pipeline([('polyfeatures', PolynomialFeatures(degree=2, include_bias=False)), ('selector', SequentialFeatureSelector(LinearRegression())),
                         ('regression', LinearRegression())])

In [59]:
selector_pipe = Pipeline([('polyfeatures', PolynomialFeatures(degree=3, include_bias=False)), ('selector', SequentialFeatureSelector(Ridge())),
                         ('ridge', Ridge())])
selector_pipe

Pipeline(steps=[('polyfeatures',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('selector', SequentialFeatureSelector(estimator=Ridge())),
                ('ridge', Ridge())])

In [60]:
#param_dict = {'ridge__alpha': [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0]}
param_dict = {'selector__n_features_to_select':[3,4,5,6,7]}

In [61]:
X_train, X_test, y_train, y_test = train_test_split(v1[list(v1.columns)[1:]], v1[['price']], shuffle=False)

In [62]:
y_test

,price
311209,13995
311210,46995
311211,73500
311267,12900
311290,17650
...,...
426788,12995
426792,32999
426793,15999
426833,6800


In [63]:
grid_2 = GridSearchCV(pipe, param_grid=param_dict)
grid_2.fit(X_train, y_train)
model_2_train_mse = mean_squared_error(y_train, grid_2.predict(X_train))
model_2_test_mse = mean_squared_error(y_test, grid_2.predict(X_test))
model_2_best_alpha = grid_2.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packages/skl

In [64]:
model_2_best_alpha

{'selector__n_features_to_select': 3}

In [71]:
grid_2.best_estimator_.steps[2][1].coef_

array([[-6.39429968e+04, -3.29804236e-02,  1.60859977e+01]])

In [72]:
grid_2.best_estimator_.steps[1][1].get_feature_names_out()

array(['x0', 'x1', 'x2'], dtype=object)

In [75]:
grid_2.best_estimator_.steps[0][1].get_feature_names_out()

array(['year', 'odometer', 'year^2', 'year odometer', 'odometer^2'],
      dtype=object)

In [76]:
print(y_test[:10], (grid_2.predict(X_test))[:10])

        price
311209  13995
311210  46995
311211  73500
311267  12900
311290  17650
311337   8995
311360   4250
311447  35995
311448   9995
311449  19995 [[15443.83190324]
 [21214.9133743 ]
 [10042.61959742]
 [ 2093.91906501]
 [15957.12243155]
 [ 8926.62143539]
 [11267.72240303]
 [21050.53599145]
 [ 6776.38066176]
 [20295.08640922]]


In [77]:
print(model_2_test_mse**0.5, model_2_train_mse**0.5)

12616.985274365057 12454.969630252337


In [78]:
v = pd.read_csv('vehicles.csv')
v = v.dropna()
v.head()


,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
126,7305672709,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,68472.0,clean,automatic,1GCWGAFP8J1309579,rwd,full-size,van,white,al
127,7305672266,auburn,0,2019.0,chevrolet,express cargo van,like new,6 cylinders,gas,69125.0,clean,automatic,1GCWGAFP4K1214373,rwd,full-size,van,white,al
128,7305672252,auburn,0,2018.0,chevrolet,express cargo van,like new,6 cylinders,gas,66555.0,clean,automatic,1GCWGAFPXJ1337903,rwd,full-size,van,white,al
215,7316482063,birmingham,4000,2002.0,toyota,echo,excellent,4 cylinders,gas,155000.0,clean,automatic,JTDBT123520243495,fwd,compact,sedan,blue,al
219,7316429417,birmingham,2500,1995.0,bmw,525i,fair,6 cylinders,gas,110661.0,clean,automatic,WBAHD6322SGK86772,rwd,mid-size,sedan,white,al


In [92]:
itms = ['manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel','region','title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'state']


mbs = cat_to_memind(v['condition'])
#mbs1 = cat_to_memind(v['type'])
for mb in mbs:
  print(mb[1])
  v[mb[1]] = mb[0]
#for mb in mbs1:
  #print(mb[1])
  #v[mb[1]] = mb[0]


salvage
good
fair
like new
new
excellent


In [93]:
v1 = v.drop(['manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel','region','title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'state', 'VIN','id'], axis=1)
v1.head()

,price,year,odometer,salvage,good,fair,like new,new,excellent
126,0,2018.0,68472.0,0,0,0,1,0,0
127,0,2019.0,69125.0,0,0,0,1,0,0
128,0,2018.0,66555.0,0,0,0,1,0,0
215,4000,2002.0,155000.0,0,0,0,0,0,1
219,2500,1995.0,110661.0,0,0,1,0,0,0


In [37]:
len(v1)

34868

In [38]:
v11 = v1.query("truck == 1")

In [39]:
len(v11)

7692

In [ ]:
list(v1.columns[1:3])

['year', 'odometer']

In [94]:
X_train, X_test, y_train, y_test = train_test_split(v1[list(v1.columns)[:-1]], v1[['price']], shuffle=False)

In [95]:
len(X_train.columns)

8

In [96]:
param_dict = {'selector__n_features_to_select':[2,3,4,5,6]}

In [98]:
lr = LinearRegression()
lr.fit(X_train, y_train)
model_2_train_mse = mean_squared_error(y_train, lr.predict(X_train))
model_2_test_mse = mean_squared_error(y_test, lr.predict(X_test))

In [100]:
lr.coef_

array([[ 1.00000000e+00, -5.03078892e-15, -7.75099410e-18,
        -1.65531872e-13, -8.71552731e-14, -3.89539769e-14,
        -6.23360780e-13,  2.52434242e-13]])

In [102]:
print(y_test[:10], (lr.predict(X_test))[:10])

        price
311209  13995
311210  46995
311211  73500
311267  12900
311290  17650
311337   8995
311360   4250
311447  35995
311448   9995
311449  19995 [[13995.]
 [46995.]
 [73500.]
 [12900.]
 [17650.]
 [ 8995.]
 [ 4250.]
 [35995.]
 [ 9995.]
 [19995.]]


In [103]:
print(model_2_test_mse**0.5, model_2_train_mse**0.5)

6.999471955388689e-12 6.908516033019866e-12


In [104]:
y_test.mean()

,0
price,16593.487438


In [105]:
y_train.mean()

,0
price,15585.433215
